In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from astropy.coordinates import SkyCoord
import os
import sys
sys.path.insert(0, '/home/aew492/lss-dipoles')
import tools
from Secrest_dipole import SecrestDipole
from multipoles import construct_templates, fit_multipole
import dipole

In [ ]:
# construct a healpix map with uncorrelated (white) noise
NSIDE = 64
noisemap = np.random.uniform(0, 1, size=hp.nside2npix(NSIDE))

# plot
hp.mollview(noisemap, title='White noise', sub=(1,2,1), min=0, max=1)

In [ ]:
# compute the power spectrum for a few low ells and figure out in which plot it is flat

In [ ]:
def compute_Cells(amps):
    """
    Returns the power C(ell) for several ells given a list of amplitudes corresponding to the a_lm coefficients
    for each ell, increasing from ell=0.
    """
    ell = 0
    i1 = 0
    Cells = np.array([])
    while i1 < len(amps):
        i2 = i1 + 2 * ell + 1
        assert i2 <= len(amps)
        Cell = compute_Cell(amps[i1:i2])
        Cells = np.append(Cells, Cell)
        ell += 1
        i1 = i2
    return Cells

In [ ]:
def compute_Cell(alms):
    """
    Returns the power C(ell) given a list of coefficients a_lm for a single ell.
    """
    assert alms.ndim <= 1
    # pad if aellems is a scalar:
    if alms.ndim == 0:
        alms = alms[..., np.newaxis]
    # infer ell from the number of moments 2ell+1
    ell = (len(alms) - 1) // 2
    assert np.mean(alms**2) == np.sum(alms**2)/(2*ell+1)
    return np.mean(alms**2)

In [ ]:
# simultaneously fit all low-ell amplitudes
ells = np.arange(0, 15)

templates = construct_templates(ells, NSIDE=NSIDE)
    
pars, stderr = fit_multipole(noisemap, templates)

In [ ]:
# compute Cells, divide by monopole to make dimensionless
Cells = compute_Cells(pars/pars[0])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells[1:], Cells[1:], 'ks')
ax.axhline(0, c='k', lw=0.5, alpha=0.5)
ax.grid(lw=0.5, alpha=0.5)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\hat{C}_\ell\,/\hat{C}_0$')
ax.set_title(f'White noise low-$\ell$ power spectrum')

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells[1:], ells[1:] * (ells[1:] + 1) * Cells[1:], 'ks')
ax.axhline(0, c='k', lw=0.5, alpha=0.5)
ax.grid(lw=0.5, alpha=0.5)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\ell\,(\ell +1)\,\hat{C}_\ell\,/\hat{C}_0$')
ax.set_title(f'White noise low-$\ell$ power spectrum')